In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import json

In [2]:
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_1.csv')

In [3]:
nan_df = main_data_df[main_data_df['도로명주소'].isna()]
nan_df[['지번주소', '도로명주소']].isna().sum()

지번주소         0
도로명주소    46345
dtype: int64

In [4]:
addr_to_download = nan_df['지번주소'].unique().tolist()
len(addr_to_download)

1650

In [27]:
# download coordinates from an open api

def download_roadname(addr_list):
    # addr_list is a list containing address names in land number address
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='
    headers = {"Authorization": "KakaoAK 55ee32778fd0e02df529c993ecdfe84f"}
    
    count =0
    addr_df = pd.DataFrame({'지번주소':[], '도로명주소':[]})
    for i, address in tqdm(enumerate(addr_list)):
        
        result = json.loads(str(requests.get(url + address, headers=headers).text))
        
        try:
            roadname_addr = result['documents'][0]['road_address_name']
        except:
            print(address)
            raise ValueError('myerror')
        
        addr_df.loc[i,'지번주소'] = address
        addr_df.loc[i,'도로명주소'] = roadname_addr
    
    return addr_df

In [28]:
roadname_addr_df = download_roadname(addr_to_download)

4it [00:00, 14.36it/s]

서울특별시 강남구 신사동 506-7


ValueError: myerror

In [29]:
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + '서울특별시 강남구 신사동 506-7'
headers = {"Authorization": "KakaoAK 55ee32778fd0e02df529c993ecdfe84f"}
result = json.loads(str(requests.get(url, headers=headers).text))

In [30]:
result

{'documents': [],
 'meta': {'is_end': True,
  'pageable_count': 0,
  'same_name': {'keyword': '서울특별시 강남구 신사동 506-7',
   'region': [],
   'selected_region': ''},
  'total_count': 0}}

In [23]:
result['documents'][0]['road_address_name']

'서울 강남구 언주로 604'